# Project SURAKSHA : Enabling awareness

<div class = "alert alert-block alert-success">

 - <b>Version</b> : 1.0

 - <b>Authors</b> : 
    -  Anjali Muralidharan<br>
    -  Chitra Nair<br>
    -  Kavish Jhaveri<br>
    -  Simantini Ghosh<br>
    -  Sonal Rai
            
-  Built in association with Indian School of Business,Hyderbad as part of Capstone Project
</div>

### Problem Description:

To create a prototype for a mobile app that ensures women & child safety though the following modules
1.	Share Live Location & Details - With a chosen few, integrate with cab giants to link and display driver details to the chosen person receiving live updates
2.	Map the City Down Based On Safety Index - Provide real time alerts when in a zone/area with a poor safety rating, citations on prior crimes committed
3.	SOS Functionality: App Profile to Integrate with Women & Police Helplines Based on subject's zip code and make calls in need
4.	Offline Mode: The app must share data while offline (TBD)
5.	Prescriptive Prompts !!!: Must deliver real time alerts, if the subject is in an area that is inhabited (by criminal records) by convicted sex offender, child trafficker or other associated crimes with convicts personal identification indicators (Photo etc.) to enable educated decision making.

<div class = "alert alert-block alert-info">
<b>This code snippet is for building a prediction model using bagging</b>
</div>

#### Loading relevant libraries

In [59]:
import pandas as pd
import numpy as np

In [60]:
import warnings
warnings.filterwarnings('ignore')

#### Loading the Delhi crime data

In [61]:
delhi_crime_data = pd.read_excel("Delhi_crime_data.xls") 
delhi_crime_data

,Address,Area,City,Category,Day Quarter Group,Day Time,Delhi Districts Cluster,Delhi Incidents Cluster,Id,Incident Category,...,weekday,quarter,Delhi_Cluster_code,Delhi_Cluster_code_encode,time_hour,crime_code,Severity_index,Safety_index,Safety_index_code,Safety_index_code_val
0,"Minto Bridge Colony, Barakhamba, New Delhi, De...",Barakhamba,New Delhi,Verbal Abuse,Quarter 1 : 12 Midnight to 6 AM,1 AM-2 AM,Low: <100 Incidents,Mid: 20-50 Incidents,3887,Catcalls/Whistles,...,5,2,Med,1,1,6,Med,0.038469,Yellow,1
1,"Minto Bridge Colony, Barakhamba, New Delhi, De...",Barakhamba,New Delhi,Non-Verbal Abuse,Quarter 1 : 12 Midnight to 6 AM,1 AM-2 AM,Low: <100 Incidents,Mid: 20-50 Incidents,3996,Ogling/Lewd Facial Expressions/Staring,...,4,2,Med,1,1,4,Med,0.038469,Yellow,1
2,"Minto Bridge Colony, Barakhamba, New Delhi, De...",Barakhamba,New Delhi,Physical Abuse,Quarter 1 : 12 Midnight to 6 AM,1 AM-2 AM,Low: <100 Incidents,Mid: 20-50 Incidents,4000,Touching /Groping,...,4,2,Med,1,1,15,High,0.038469,Yellow,1
3,"Universitas Delhi, Minto Bridge Colony, Barakh...",Barakhamba,New Delhi,Non-Verbal Abuse,Quarter 1 : 12 Midnight to 6 AM,1 AM-2 AM,Low: <100 Incidents,Mid: 20-50 Incidents,2893,Taking pictures without permission,...,6,2,Med,1,1,5,Med,0.038469,Yellow,1
4,"Gandhi Market Area, Barakhamba, New Delhi, Del...",Barakhamba,New Delhi,Verbal Abuse,Quarter 1 : 12 Midnight to 6 AM,1 AM-2 AM,Low: <100 Incidents,Mid: 20-50 Incidents,2009,Catcalls/Whistles,...,2,1,Med,1,13,6,Med,0.038469,Yellow,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5824,"Ghazipur Village, Gazipur, New Delhi, Delhi, I...",Gazipur,East Delhi,Physical Abuse,Quarter 2 : 6 AM to 12 Noon,9 AM-10 AM,Mid: 100-500 Incidents,Low: < 20 Incidents,4127,Touching /Groping,...,3,1,Low,2,9,15,High,0.001649,Green,2
5825,"Block C, Maurice Nagar, Roop Nagar, Delhi, 110...",Roop Nagar,North Delhi,Verbal Abuse,Quarter 4 : 6 PM to 12 Midnight,9 PM-10 PM,Mid: 100-500 Incidents,Low: < 20 Incidents,10774,Catcalls/Whistles,...,0,3,Low,2,21,6,Med,0.001649,Green,2
5826,"Roop Nagar, Delhi",Roop Nagar,North Delhi,Verbal Abuse,Quarter 4 : 6 PM to 12 Midnight,9 PM-10 PM,Mid: 100-500 Incidents,Low: < 20 Incidents,6135,Catcalls/Whistles,...,4,3,Low,2,21,6,Med,0.001649,Green,2
5827,"New Aruna Nagar, New Delhi, Delhi, India",New Aruna Nagar,North Delhi,Physical Abuse,Quarter 2 : 6 AM to 12 Noon,9 AM-10 AM,Mid: 100-500 Incidents,Low: < 20 Incidents,1059,Touching /Groping,...,1,3,Low,2,9,15,High,0.001649,Green,2


In [62]:
delhi_crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5829 entries, 0 to 5828
Data columns (total 32 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Address                    5829 non-null   object        
 1   Area                       5829 non-null   object        
 2   City                       5829 non-null   object        
 3   Category                   5829 non-null   object        
 4   Day Quarter Group          5829 non-null   object        
 5   Day Time                   5829 non-null   object        
 6   Delhi Districts Cluster    5829 non-null   object        
 7   Delhi Incidents Cluster    5829 non-null   object        
 8   Id                         5829 non-null   int64         
 9   Incident Category          5829 non-null   object        
 10  Incident Date              5829 non-null   datetime64[ns]
 11  Incident Time              5829 non-null   datetime64[ns]
 12  Title 

In [64]:
crime_subset = delhi_crime_data[['Area','year','month','day','dayofweek','time_hour','dayofyear','crime_code','Category']]
crime_subset.head(5) #'Category','Severity_index',,'Safety_index'

,Area,year,month,day,dayofweek,time_hour,dayofyear,crime_code,Category
0,Barakhamba,2014,4,19,5,1,109,6,Verbal Abuse
1,Barakhamba,2014,4,4,4,1,94,4,Non-Verbal Abuse
2,Barakhamba,2014,4,18,4,1,108,15,Physical Abuse
3,Barakhamba,2014,4,13,6,1,103,5,Non-Verbal Abuse
4,Barakhamba,2014,2,5,2,13,36,6,Verbal Abuse


In [65]:
enc = crime_subset[['Area','year','month','day','dayofweek','time_hour','dayofyear','Category']]

In [66]:
encoded_data = pd.get_dummies(enc, columns = ['Area','year','month','day','dayofweek','time_hour','dayofyear','Category'])
encoded_data.head(5) #'Category',,'Severity_index'

,Area_AIIMS,Area_Adarsh Nagar,Area_Adchini,Area_Ajmeri Gate,Area_Akbar Road,Area_Alaknanda,Area_Alipur,Area_Anand Parbat,Area_Anand Vihar,Area_Andrews Ganj,...,dayofyear_359,dayofyear_360,dayofyear_361,dayofyear_362,dayofyear_363,dayofyear_366,Category_Non-Verbal Abuse,Category_Others,Category_Physical Abuse,Category_Verbal Abuse
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [67]:
data = pd.concat([encoded_data,crime_subset['crime_code']],axis=1)
data

,Area_AIIMS,Area_Adarsh Nagar,Area_Adchini,Area_Ajmeri Gate,Area_Akbar Road,Area_Alaknanda,Area_Alipur,Area_Anand Parbat,Area_Anand Vihar,Area_Andrews Ganj,...,dayofyear_360,dayofyear_361,dayofyear_362,dayofyear_363,dayofyear_366,Category_Non-Verbal Abuse,Category_Others,Category_Physical Abuse,Category_Verbal Abuse,crime_code
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,15
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,5
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,15
5825,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
5826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6
5827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,15


In [68]:
from sklearn.preprocessing import StandardScaler

In [69]:
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(crime_subset['crime_code'])
# data_scaled

#### Feature set selection

In [70]:
x_features =['Area','year','month','day','dayofweek','time_hour','dayofyear','Category'] #'Category',,'Severity_index'

#### Setting x and y variables

In [71]:
#setting X and Y:
# X = delhi_crime_data[x_features]
# y = delhi_crime_data['crime_code']

#setting X and Y:
X = data.drop('crime_code',axis='columns')
y = data['crime_code']

### Partitioning data to train and test sets

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 21)

In [74]:
X_train.shape

(4371, 668)

In [75]:
X_test.shape

(1458, 668)

In [76]:
from sklearn.tree import DecisionTreeClassifier

In [77]:
from sklearn.model_selection import cross_val_score

In [78]:
scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores

array([0.48627787, 0.2890223 , 0.22727273, 0.4193825 , 0.50042918])

In [79]:
scores.mean()

0.38447691752736696

In [80]:
from sklearn.ensemble import BaggingClassifier

In [81]:
bag_model = BaggingClassifier(
base_estimator=DecisionTreeClassifier(), 
n_estimators=100, 
max_samples=0.8, 
bootstrap=True,
oob_score=True,
random_state=0
)

In [82]:
bag_model.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.8,
                  n_estimators=100, oob_score=True, random_state=0)

In [83]:
bag_model.oob_score_

0.6318920155570807

In [84]:
bag_model.score(X_test, y_test)

0.6227709190672154

# ****